# `Backend code`

- Colab - Vscode 연동
- colab 보안키 설정(NGROK, OPENAI)
- Public URL(https://xxx-xx-xxx-xx-xx.ngrok-free.app) Frontend .env 파일에 저장

In [ ]:
# 필요한 라이브러리 설치
!pip install fastapi uvicorn pydub transformers torch numpy python-dotenv aiohttp pyngrok openai nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 1.9 MB/s eta 0:00:00


In [ ]:
# 코드 셀
import os
import logging
from dotenv import load_dotenv
import torch
from transformers import AutoProcessor, MusicgenForConditionalGeneration
import numpy as np
from pydub import AudioSegment
import io
from fastapi import FastAPI, HTTPException, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import FileResponse
from pydantic import BaseModel, Field
from typing import Optional, Tuple
import uuid
from pyngrok import ngrok
import asyncio
import aiohttp
from openai import AsyncOpenAI
import nest_asyncio
import uvicorn

In [ ]:
# colab 보안 비밀 키 엑세스
from google.colab import userdata
ngrok_auth_token = userdata.get('NGROK_AUTH_TOKEN')
ngrok.set_auth_token(ngrok_auth_token)
openai_api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
# FastAPI 앱 설정
app = FastAPI()

# CORS 미들웨어 설정
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # 모든 오리진 허용 (프로덕션에서는 특정 오리진만 허용하도록 변경해야 함)
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# OpenAI 클라이언트 초기화
openai_client = AsyncOpenAI(api_key=openai_api_key)

# 모델 및 프로세서 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small").to(device)

# 요청 및 응답 모델
class MusicRequest(BaseModel):
    user_input: str = Field(..., min_length=1, max_length=500)
    duration: int = Field(10, ge=5, le=30)

class TaskStatus(BaseModel):
    task_id: str
    status: str
    message: Optional[str] = None
    progress: Optional[int] = None

# 태스크 저장소
tasks = {}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/7.87k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/usr/local/lib/python3.10/dist-packages/transformers/models/encodec/modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


generation_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

In [ ]:
# 최적화된 프롬프트 생성 함수
async def generate_optimized_prompt(user_input: str) -> str:
    try:
        response = await openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are an AI music composer assistant."},
                {"role": "user", "content": f"Create a detailed music generation prompt based on this input: {user_input}"}
            ],
            max_tokens=100,
            temperature=0.7
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        logger.error(f"Error in generate_optimized_prompt: {str(e)}")
        raise

# 음악 생성 함수
async def generate_music(prompt: str, duration: int = 10) -> AudioSegment:
    inputs = processor(
        text=[prompt],
        padding=True,
        return_tensors="pt",
    ).to(device)

    audio_values = model.generate(
        **inputs,
        do_sample=True,
        guidance_scale=3,
        max_new_tokens=256
    )

    audio_data = audio_values[0, 0].cpu().numpy()
    audio_data = np.int16(audio_data / np.max(np.abs(audio_data)) * 32767)

    audio_segment = AudioSegment(
        audio_data.tobytes(),
        frame_rate=sampling_rate,
        sample_width=2,
        channels=1
    )

    if len(audio_segment) < duration * 1000:
        audio_segment = audio_segment * (duration * 1000 // len(audio_segment) + 1)
    audio_segment = audio_segment[:duration * 1000]

    return audio_segment

# 음악 생성 프로세스
async def process_music_generation(task_id: str, user_input: str, duration: int):
    try:
        tasks[task_id]["status"] = "processing"
        tasks[task_id]["progress"] = 10

        audio_segment = await generate_music(user_input, duration)
        tasks[task_id]["progress"] = 90

        file_path = f"/content/generated_audio_{task_id}.wav"
        audio_segment.export(file_path, format="wav")

        tasks[task_id]["status"] = "completed"
        tasks[task_id]["message"] = "Music generation completed"
        tasks[task_id]["progress"] = 100
    except Exception as e:
        logger.error(f"Error in process_music_generation: {str(e)}")
        tasks[task_id]["status"] = "failed"
        tasks[task_id]["message"] = str(e)

In [ ]:
# API 엔드포인트
@app.post("/api/generate-music")
async def generate_music_endpoint(request: MusicRequest, background_tasks: BackgroundTasks):
    task_id = str(uuid.uuid4())
    tasks[task_id] = {"status": "pending", "progress": 0}
    background_tasks.add_task(process_music_generation, task_id, request.user_input, request.duration)
    return {"task_id": task_id}

@app.get("/api/task/{task_id}", response_model=TaskStatus)
async def get_task_status(task_id: str):
    if task_id not in tasks:
        raise HTTPException(status_code=404, detail="Task not found")
    return TaskStatus(task_id=task_id, **tasks[task_id])

@app.get("/api/download/{task_id}")
async def download_music(task_id: str):
    if task_id not in tasks or tasks[task_id]["status"] != "completed":
        raise HTTPException(status_code=404, detail="Generated music not found")
    file_path = f"/content/generated_audio_{task_id}.wav"
    return FileResponse(file_path, media_type="audio/wav", filename=f"generated_music_{task_id}.wav")

# 서버 실행
if __name__ == "__main__":
    nest_asyncio.apply()

    # ngrok 터널 생성
    ngrok_tunnel = ngrok.connect(8000)
    print('Public URL:', ngrok_tunnel.public_url)

    async def run_server():
        config = uvicorn.Config(app, host="0.0.0.0", port=8000)
        server = uvicorn.Server(config)
        await server.serve()

    asyncio.get_event_loop().run_until_complete(run_server())

INFO:     Started server process [1317]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://3b2c-35-230-49-92.ngrok-free.app
